In [1]:
'''
Mulliken population analysis with IAO orbitals
'''

import numpy
from functools import reduce
from sys import stdout
from pyscf import gto, scf, lo
from pyscf.tools.dump_mat import dump_rec
from pyscf.tools.dump_mat import dump_mo

mol = gto.M(atom="""
  C1    0.0000000   -0.0000000   -0.7595848
  H1    0.5222891   -0.9046312   -1.1361620
  H1    0.5222891    0.9046312   -1.1361620
  H1   -1.0445781    0.0000000   -1.1361620
  C2   -0.0000000    0.0000000    0.7595848
  H2    1.0445781   -0.0000000    1.1361620
  H2   -0.5222891   -0.9046312    1.1361620
  H2   -0.5222891    0.9046312    1.1361620
""", basis='cc-pvdz')

mol.build()

nao_h2o = mol.nao // 2

h2o_1_idx = gto.search_ao_label(mol, ["C1", "H1"])
h2o_2_idx = gto.search_ao_label(mol, ["C2", "H2"])

print(h2o_1_idx)
print(h2o_2_idx)

mf = scf.RHF(mol).run()
c_ao_lo = lo.orth_ao(mf, 'nao')

assert numpy.linalg.norm(reduce(numpy.dot, [c_ao_lo.T, mf.get_ovlp(), c_ao_lo]) - numpy.eye(c_ao_lo.shape[0])) < 1e-8

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28]
[29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 55 56 57]
converged SCF energy = -79.2329735197058


In [2]:
# check the localality
from scipy.linalg import sqrtm
ww = numpy.einsum("mn,np->mp", sqrtm(mf.get_ovlp()), c_ao_lo)
w2 = ww**2

for i in range(nao_h2o):    
    assert abs(1.0 - sum(w2[h2o_1_idx, i])) < 1e-1
    assert abs(sum(w2[h2o_2_idx, i])) < 1e-1
    assert abs(1.0 - sum(w2[h2o_2_idx, i + nao_h2o])) < 1e-1
    assert abs(sum(w2[h2o_1_idx, i + nao_h2o])) < 1e-1

In [3]:
# transform mo_occ to IAO representation. Note the AO dimension is reduced
ovlp_ao = mf.get_ovlp()
dm_ao = mf.make_rdm1()
dm_lo = reduce(numpy.dot, [c_ao_lo.T, ovlp_ao, dm_ao, ovlp_ao, c_ao_lo])
assert numpy.linalg.norm(reduce(numpy.dot, [dm_lo/2, dm_lo/2]) - dm_lo/2) < 1e-8

idx_a = h2o_1_idx
idx_b = h2o_2_idx

dm_aa = dm_lo[numpy.ix_(idx_a, idx_a)]
dm_bb = dm_lo[numpy.ix_(idx_b, idx_b)]
dm_ab = dm_lo[numpy.ix_(idx_a, idx_b)]

In [4]:
from scipy import linalg

uab, sab, vhab = linalg.svd(dm_ab)
numpy.linalg.norm(
    dm_ab - reduce(numpy.dot, [uab, numpy.diag(sab), vhab])
)

1.6022421332888106e-15

In [5]:
from pyscf import tools

ua, sa, vha = linalg.svd(dm_aa)
numpy.linalg.norm(
    dm_aa - reduce(numpy.dot, [ua, numpy.diag(sa), vha])
)

print(sa)
# dump_rec(stdout, uab)
# dump_rec(stdout, ua)

ub, sb, vhb = linalg.svd(dm_bb)
numpy.linalg.norm(
    dm_bb - reduce(numpy.dot, [ub, numpy.diag(sb), vhb])
)

print(sb)
# dump_rec(stdout, ub)
# dump_rec(stdout, vhab.T)

[1.99999980e+00 1.99795251e+00 1.98666721e+00 1.98666721e+00
 1.00000000e+00 1.33327918e-02 1.33327903e-02 2.04748988e-03
 2.02304956e-07 1.17824835e-15 7.25921140e-16 5.34316665e-16
 1.99840022e-16 1.99840022e-16 1.99840022e-16 1.99840022e-16
 1.99840022e-16 1.99840022e-16 1.99840022e-16 1.99840022e-16
 1.99840022e-16 1.99840022e-16 1.99840022e-16 1.99840022e-16
 1.99840022e-16 1.99840022e-16 1.99840022e-16 1.99840022e-16
 1.26429141e-16]
[1.99999980e+00 1.99795251e+00 1.98666721e+00 1.98666721e+00
 1.00000000e+00 1.33327918e-02 1.33327903e-02 2.04748988e-03
 2.02304956e-07 6.42998133e-16 5.22128406e-16 4.70973288e-16
 1.99840022e-16 1.99840022e-16 1.99840022e-16 1.99840022e-16
 1.99840022e-16 1.99840022e-16 1.99840022e-16 1.99840022e-16
 1.99840022e-16 1.99840022e-16 1.99840022e-16 1.99840022e-16
 1.99840022e-16 1.99840022e-16 1.99840022e-16 1.43597562e-16
 7.97745507e-17]


In [6]:
c_ao_lo_a = c_ao_lo[:,idx_a]
c_pio_a   = reduce(numpy.dot, [c_ao_lo_a, ua])
npio = c_ao_lo_a.shape[1]

for pp in range(npio):
    tools.cubegen.orbital(mol, f'./cube/ch3_a_{pp}_{sa[pp]:4.2e}.cube', c_pio_a[:,pp])

c_ao_lo_b = c_ao_lo[:,idx_b]
c_pio_b   = reduce(numpy.dot, [c_ao_lo_b, ub])
npio = c_ao_lo_b.shape[1]

for pp in range(npio):
    tools.cubegen.orbital(mol, f'./cube/ch3_b_{pp}_{sb[pp]:4.2e}.cube', c_pio_b[:,pp])

In [7]:
assert numpy.linalg.norm(
    reduce(numpy.dot, [c_pio_a.T, ovlp_ao, c_pio_a]) - numpy.eye(c_pio_a.shape[1])
)
dump_mo(mol, c_pio_a)

               #0        #1        #2        #3        #4       
 0 C1 1s      -1.00241  -0.01357   0.00000  -0.00000  -0.10585
 0 C1 2s      -0.00214   0.30511  -0.00000   0.00000  -0.54012
 0 C1 3s       0.01077   0.19405  -0.00000  -0.00000  -0.16626
 0 C1 2px     -0.00000  -0.00000  -0.42697   0.00000  -0.00000
 0 C1 2py     -0.00000  -0.00000   0.00000   0.42697  -0.00000
 0 C1 2pz     -0.00145  -0.23681   0.00000  -0.00000  -0.75849
 0 C1 3px      0.00000  -0.00000  -0.18450   0.00000   0.00000
 0 C1 3py     -0.00000  -0.00000   0.00000   0.18450  -0.00000
 0 C1 3pz     -0.00009  -0.11628   0.00000  -0.00000  -0.25436
 0 C1 3dxy    -0.00000  -0.00000   0.00000   0.02269  -0.00000
 0 C1 3dyz     0.00000   0.00000  -0.00000  -0.01429   0.00000
 0 C1 3dz^2    0.00017  -0.01762   0.00000  -0.00000  -0.05965
 0 C1 3dxz    -0.00000   0.00000   0.01429  -0.00000  -0.00000
0 C1 3dx2-y2  -0.00000   0.00000   0.02269  -0.00000   0.00000
 1 H1 1s       0.00164   0.22498  -0.18776  -0.32521 

In [8]:
fock_ao = mf.get_fock()
fock_pio_a = reduce(numpy.dot, [c_pio_a.T, fock_ao, c_pio_a])
dump_rec(stdout, fock_pio_a)

        #0        #1        #2        #3        #4       
0     -11.22354   0.08009  -0.00000  -0.00000  -0.00281
1       0.08009  -0.81004   0.00000  -0.00000   0.12929
2      -0.00000   0.00000  -0.53089   0.00000  -0.00000
3      -0.00000  -0.00000   0.00000  -0.53089   0.00000
4      -0.00281   0.12929  -0.00000   0.00000  -0.02908
5       0.00000  -0.00000  -0.00000  -0.00061  -0.00000
6       0.00000   0.00000   0.00061  -0.00000   0.00000
7       0.00012   0.00324  -0.00000   0.00000   0.05514
8      -0.00004   0.00197  -0.00000   0.00000   0.15111
9       0.00000  -0.00040   0.00107   0.00123  -0.01175
10     -0.00000   0.00005   0.00114  -0.00094   0.00450
11     -0.00001   0.00061  -0.00030   0.00054   0.00970
12      0.00002  -0.00070  -0.00156  -0.00111  -0.06804
13      0.00000  -0.00131  -0.00067   0.00009   0.00254
14     -0.00001  -0.00020  -0.00230  -0.00126  -0.00155
15      0.00000  -0.00010   0.00156   0.00116  -0.03130
16     -0.00001  -0.00059   0.00117  -0.00122 

In [9]:
c_ao_lo_a = c_ao_lo[:,idx_a]
c_pio_a   = reduce(numpy.dot, [c_ao_lo_a, ua.T])
npio = c_ao_lo_a.shape[1]

for pp in range(npio):
    tools.cubegen.orbital(mol, f'./cube/h2o_a_{pp}_{s[pp]:4.2f}_t.cube', c_pio_a[:,pp])

c_ao_lo_b = c_ao_lo[:,idx_b]
c_pio_b   = reduce(numpy.dot, [c_ao_lo_b, ub.T])
npio = c_ao_lo_b.shape[1]

for pp in range(npio):
    tools.cubegen.orbital(mol, f'./cube/h2o_b_{pp}_{s[pp]:4.2f}_t.cube', c_pio_b[:,pp])

NameError: name 's' is not defined